<a href="https://colab.research.google.com/github/tdiego74/Clase_Data_Science_III_PBLC/blob/main/Proyecto_DS_III_Entrega_Final_Tocci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Proyecto MeLiSA - Reseñas en español de Mercado Libre**

##**Abstracto con Motivación y Audiencia**

En este proyecto se analiza el dataset MeLiSA (Mercado Libre Sentiment Analysis) para explorar patrones de texto en reseñas de productos de Mercado Libre. Este trabajo está orientado a aquellos interesados en el procesamiento de lenguaje natural (NLP) y en cómo las reseñas pueden ser analizadas para comprender el sentimiento del usuario, ayudando a mejorar experiencias comerciales y estrategias de negocio.

El objetivo final es demostrar el flujo completo de análisis: desde la limpieza y el preprocesamiento del texto hasta la implementación de técnicas avanzadas de análisis exploratorio, visualización y el desarrollo de un modelo predictivo basado en redes neuronales.

##**Contexto Comercial y Analítico**

Mercado Libre es una plataforma de comercio electrónico líder en América Latina. Las reseñas de los productos proporcionadas por los usuarios son una fuente invaluable de información sobre las preferencias del consumidor, la calidad del producto y la satisfacción general.

El análisis de estas reseñas no solo ayuda a los vendedores a entender mejor a sus clientes, sino que también mejora la experiencia de compra al resaltar los productos más valorados y detectar posibles problemas antes de que se conviertan en una tendencia.

##**Preguntas/Hipótesis a Resolver**

- ¿Qué palabras o frases son más comunes en las reseñas positivas y negativas?
- ¿Es posible clasificar automáticamente las reseñas según su calificación o sentimiento?
- ¿Qué características textuales predominan en las reseñas según su longitud, estilo o estructura lingüística?

##**Objetivo**

El objetivo principal de este trabajo es demostrar el uso de técnicas de procesamiento de lenguaje natural (NLP) para analizar y extraer patrones significativos de un conjunto de reseñas. Este proceso incluye:
- Preprocesamiento del texto.
- Análisis exploratorio para identificar patrones significativos en las palabras y estructuras lingüísticas.
- Entrenamiento de un modelo basado en redes neuronales para predecir el sentimiento de las reseñas.
